In [1]:
import re
import pandas as pd
from textblob import TextBlob
from textblob.exceptions import NotTranslated

FILE_NAME = 'inter_tweets.csv'

In [2]:
company_tweets = pd.read_csv(FILE_NAME, sep=',', names=['id', 'text'])
company_tweets.drop_duplicates(subset='text', inplace=True)
company_tweets.sort_values('id', inplace=True)

In [3]:
company_tweets_clean = pd.DataFrame(columns=('id', 'date', 'text', 'text_en', 'sentiment_polarity', 'sentiment_subjectivity'))

for i in company_tweets.index:
    if company_tweets.at[i, 'id'] > 3560:
        full_text = company_tweets.at[i, 'text']
        date, tweet = full_text.split("Text=")
        date_splitted = date.split(' ')
        
        if date_splitted[1] == "Sep":
            new_date_format = "2019-09-{:s} {:s}".format(date_splitted[2], date_splitted[3])
        elif date_splitted[1] == "Oct":
            new_date_format = "2019-10-{:s} {:s}".format(date_splitted[2], date_splitted[3])
        else:
            new_date_format = "2019-11-{:s} {:s}".format(date_splitted[2], date_splitted[3])
        
        tweet = ''.join(tweet.split('\\n'))
        clean_tweet = ' '.join(re.sub("(@[A-Za-z0-9-ZÀ-ÿ]+)|([^0-9A-Za-z-ZÀ-ÿ \t])|(\w+://\S+)", " ", tweet).split())
        
        phrase = TextBlob(clean_tweet)
        try:
            translate = TextBlob(str(phrase.translate(from_lang='pt', to='en_US')))
            single_row = {
                         'id': company_tweets.at[i, 'id'],
                         'date': new_date_format,
                         'text': str(phrase),
                         'text_en': str(translate),
                         'sentiment_polarity': translate.sentiment.polarity,
                         'sentiment_subjectivity': translate.sentiment.subjectivity
                          }
            company_tweets_clean = company_tweets_clean.append(single_row, ignore_index=True)
        except NotTranslated:
                print("Invalid tweet {:s}!".format(str(phrase)))

Invalid tweet 2 2!
Invalid tweet 1 2!
Invalid tweet RT 1 2!
Invalid tweet RT 1 2!
Invalid tweet 3!


HTTPError: HTTP Error 429: Too Many Requests

In [4]:
final = pd.read_csv(FILE_NAME.split('.')[0] + "_clean.csv", sep=',')
final.drop(final.columns[[0]], axis=1, inplace=True)

In [5]:
merged_data = pd.concat([final, company_tweets_clean])
print(merged_data.to_string())

        id                 date                                               text                                            text_en  sentiment_polarity  sentiment_subjectivity
0        1  2019-09-10 00:20:52  O Banco Inter deveria lançar uma maquinina de ...  Banco Inter should launch a card machine for P...            0.000000                0.000000
1        2  2019-09-10 00:44:44                Uso a do Banco Inter Acho tranquilo                    I use Inter Bank I find it easy            0.433333                0.833333
2        3  2019-09-10 00:45:08  Banco Inter Foi a melhor opção pra mim conta s...  Banco Inter It was the best option for me acco...            0.600000                0.250000
3        4  2019-09-10 00:50:26  assim como já comentado por outros a do banco ...  as already commented by others the bank Inter ...            0.106667                0.448571
4        5  2019-09-10 00:51:04  Estou usando Banco Inter há pouco tempo Foi rá...  I have been using Inter Ba

In [6]:
merged_data.to_csv(FILE_NAME.split('.')[0] + "_clean.csv", sep=',', encoding='utf-8')
